In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('/content/sample_data/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [ ]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [ ]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [ ]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [ ]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,True,False,False,True,False
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,False,False,True,True,False
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,True,False,False,True,False
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,True,False,False,True,False
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,False,False,True,True,False


In [ ]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

In [ ]:
X_train.shape

(8000, 13)

In [ ]:
X_test.shape

(2000, 13)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

BUILD THE MODEL(ANN)

In [ ]:
def ANN(X_train, y_train, X_test, y_test, loss):
     # Check for and handle NaN/infinite values
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)
    y_test = np.nan_to_num(y_test, nan=0.0, posinf=0.0, neginf=0.0)

    # Convert arrays to a supported float type (if necessary)
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_test = y_test.astype(np.float32)

    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 0.4949 - accuracy: 0.7962
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4703 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4628 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4553 - accuracy: 0.7983
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4461 - accuracy: 0.8029
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.8106
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.8154
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4131 - accuracy: 0.8202
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4018 - accuracy: 0.8266
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3907 - accura

 The precision, recall and f1 score of Class 1 is very low due to imbalanced dataset

In [ ]:
count_class_0, count_class_1 = df1.Exited.value_counts()

In [ ]:
# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
Exited
0    2037
1    2037
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
y_train.value_counts()

Exited
1    1630
0    1629
Name: count, dtype: int64

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6901 - accuracy: 0.5272
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6644 - accuracy: 0.5940
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6471 - accuracy: 0.6238
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6323 - accuracy: 0.6493
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6189 - accuracy: 0.6686
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6079 - accuracy: 0.6827
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5993 - accuracy: 0.6873
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5906 - accuracy: 0.6965
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.6984
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5780 - accura

 There is considerable Improvement in the f1, recall and precision scores of Class 1 Value. The f1 score has improved from 0.59
  to 0.77.

Method 2: Oversampling

In [ ]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
Exited
0    7963
1    7963
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6390 - accuracy: 0.6411
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6004 - accuracy: 0.6797
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5728 - accuracy: 0.7033
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5590 - accuracy: 0.7158
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7280
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5303 - accuracy: 0.7403
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5165 - accuracy: 0.7488
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5054 - accuracy: 0.7551
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4939 - accuracy: 0.7608
Epoch 10/100
399/399 [==============================] - 1s 3ms/step - loss: 0.4850 - accura

The f1-score for minority class 1 improved from 0.59 to 0.77.

Method3: SMOTE

In [ ]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

Exited
1    7963
0    7963
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 3s 2ms/step - loss: 0.6524 - accuracy: 0.6179
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5930 - accuracy: 0.6828
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7179
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5271 - accuracy: 0.7419
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5060 - accuracy: 0.7562
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4938 - accuracy: 0.7629
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4860 - accuracy: 0.7651
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4798 - accuracy: 0.7685
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4749 - accuracy: 0.7691
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4711 - accura

SMOT Oversampling increases f1 score of minority class 1 from 0.59 to 0.79.

Method4: Use of Ensemble with undersampling

In [ ]:
df1.Exited.value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [ ]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [ ]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,True,False,False,False,True,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,False,True,False,True,False,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,True,False,False,True,False,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,False,True,False,False,True,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,True,False,False,False,True,0


In [ ]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [ ]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 2s 3ms/step - loss: 0.6842 - accuracy: 0.5584
Epoch 2/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6664 - accuracy: 0.6035
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6534 - accuracy: 0.6163
Epoch 4/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6414 - accuracy: 0.6326
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6308 - accuracy: 0.6486
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.6614
Epoch 7/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6139 - accuracy: 0.6650
Epoch 8/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6049 - accuracy: 0.6810
Epoch 9/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5983 - accuracy: 0.6874
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5936 - accuracy: 0.6906
Epoch 11/

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 2s 4ms/step - loss: 0.6762 - accuracy: 0.5750
Epoch 2/100
98/98 [==============================] - 0s 5ms/step - loss: 0.6435 - accuracy: 0.6429
Epoch 3/100
98/98 [==============================] - 0s 4ms/step - loss: 0.6250 - accuracy: 0.6528
Epoch 4/100
98/98 [==============================] - 0s 4ms/step - loss: 0.6138 - accuracy: 0.6675
Epoch 5/100
98/98 [==============================] - 0s 4ms/step - loss: 0.6055 - accuracy: 0.6765
Epoch 6/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5959 - accuracy: 0.6829
Epoch 7/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5883 - accuracy: 0.6922
Epoch 8/100
98/98 [==============================] - 0s 5ms/step - loss: 0.5814 - accuracy: 0.6947
Epoch 9/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5767 - accuracy: 0.7005
Epoch 10/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5696 - accuracy: 0.7043
Epoch 11/

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 2s 3ms/step - loss: 0.6823 - accuracy: 0.5574
Epoch 2/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6260
Epoch 3/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6272 - accuracy: 0.6527
Epoch 4/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6077 - accuracy: 0.6769
Epoch 5/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5929 - accuracy: 0.6863
Epoch 6/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5827 - accuracy: 0.6928
Epoch 7/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5746 - accuracy: 0.7022
Epoch 8/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5678 - accuracy: 0.7054
Epoch 9/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5622 - accuracy: 0.7083
Epoch 10/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5574 - accuracy: 0.7101
Epoch 11/

In [ ]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [ ]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.74      0.83      1593
           1       0.44      0.81      0.57       407

    accuracy                           0.76      2000
   macro avg       0.69      0.78      0.70      2000
weighted avg       0.84      0.76      0.78      2000



The f1-score for minority class 1 is 0.57